<a href="https://colab.research.google.com/github/Tanjiro00/pochti_AI/blob/master/inclass_decision_trees_and_ensembles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_blobs, make_circles, make_classification, load_iris, load_digits
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import tree, metrics, model_selection
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from collections import Counter

# Решающие деревья

### Визуализируем решаюшее дерево для знакомого датасета Iris

In [ ]:
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, random_state=42)

In [ ]:
clf = DecisionTreeClassifier(max_depth=2, random_state=42)
clf.fit(X_train, y_train)
# clf.predict(X_test)

In [ ]:
plt.figure(figsize=(12,10))
_ = tree.plot_tree(clf, 
                feature_names=iris.feature_names,  
                class_names=iris.target_names,
                filled=True)

### Посмотрим как выглядит разделяющая плоскость

#### Создадим легко разделимый датасет

In [ ]:
X, y = make_blobs(n_samples=1000, centers=2, n_features=2, random_state=42, cluster_std=3)

In [ ]:
plt.figure(figsize=(12,10))
plt.scatter(X[:,0], X[:,1], c=y)

#### Построим классификатор – как обычно, делим на трейн и тест, учим, считаем метрики

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

##### Считаем метрики -- accuracy и accuracy на кросс-валидации

In [ ]:
clf.score(X_test, y_test)

In [ ]:
metrics.accuracy_score(y_test, y_pred)


In [ ]:
np.mean(cross_val_score(clf, X, y, cv=5))

#### Посмотрим на разделяющую плоскость

In [ ]:
def get_meshgrid(data, step=.05, border=.5,):
    x_min, x_max = data[:, 0].min() - border, data[:, 0].max() + border
    y_min, y_max = data[:, 1].min() - border, data[:, 1].max() + border
    
    return np.meshgrid(np.arange(x_min, x_max, step), np.arange(y_min, y_max, step))

def plot_decision_surface(estimator, train_data, train_labels, test_data, test_labels):
    estimator.fit(train_data, train_labels)
    
    plt.figure(figsize = (16, 6))
    light_colors = ListedColormap(['lightyellow','lightcoral'])
    colors = ListedColormap(['yellow', 'red'])
    
    #plot decision surface on the train data 
    plt.subplot(1,2,1)
    xx, yy = get_meshgrid(train_data)
    mesh_predictions = np.array(estimator.predict(np.c_[xx.ravel(), yy.ravel()])).reshape(xx.shape)
    plt.pcolormesh(xx, yy, mesh_predictions, cmap=light_colors)
    plt.scatter(train_data[:, 0], train_data[:, 1], c=train_labels, s=40, cmap=colors)
    plt.title('Train data, accuracy={}'.format(metrics.accuracy_score(train_labels, estimator.predict(train_data))))
    
    #plot decision surface on the test data
    plt.subplot(1,2,2)
    plt.pcolormesh(xx, yy, mesh_predictions, cmap=light_colors)
    plt.scatter(test_data[:, 0], test_data[:, 1], c=test_labels, s=40, cmap=colors)
    plt.title('Test data, accuracy={}'.format(metrics.accuracy_score(test_labels, estimator.predict(test_data))))

In [ ]:
plot_decision_surface(clf, X_train, y_train, X_test, y_test)

#### Посмотрим на менее удачный датасет

In [ ]:
X_circles, y_circles = make_circles(n_samples=1000, noise=0.3, random_state=42)

In [ ]:
plt.figure(figsize=(12,10))
plt.scatter(X_circles[:,0], X_circles[:,1], c=y_circles)

In [ ]:
X_train_circles, X_test_circles, y_train_circles, y_test_circles = train_test_split(X_circles, y_circles, random_state=42)

In [ ]:
clf_circles = DecisionTreeClassifier(random_state=42)
clf_circles.fit(X_train_circles, y_train_circles)
predict = clf_circles.predict(X_test_circles)
print(f"Accuracy: {clf_circles.score(X_test_circles, y_test_circles)}")
print(f"Accuracy on CV: {np.mean(cross_val_score(clf_circles, X_circles, y_circles, cv=5))}")

In [ ]:
plot_decision_surface(clf_circles, X_train_circles, y_train_circles, X_test_circles, y_test_circles)

In [ ]:
clf_circles = RandomForestClassifier(n_estimators=500, random_state=42)
clf_circles.fit(X_train_circles, y_train_circles)
predict = clf_circles.predict(X_test_circles)
print(f"Accuracy: {clf_circles.score(X_test_circles, y_test_circles)}")
print(f"Accuracy on CV: {np.mean(cross_val_score(clf_circles, X_circles, y_circles, cv=5))}")

In [ ]:
plot_decision_surface(clf_circles, X_train_circles, y_train_circles, X_test_circles, y_test_circles)

### Давайте посмотрим, какие параметры решающего дерева можно менять

In [ ]:
DecisionTreeClassifier()

In [ ]:
plot_decision_surface(DecisionTreeClassifier(max_depth=200), X_train_circles, y_train_circles, X_test_circles, y_test_circles)

#### Самостоятельная работа
Изучите зависимость качества работы алгоритма на тесте при изменении параметров

1.   max_depth
2.   min_samples_leaf

Нарисуйте график

In [ ]:
# Todo: изучить качество работы алгоритма при изменении max_depth

In [ ]:
# Todo: изучить качество работы алгоритма при изменении min_samples_leaf
### Ваш код здесь

### Посмотрим на качество на обучении и кросс-валидации

In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
def draw_accuracy_graph(alg, param, grid_list, X, y):
    # Инициализируем валидацию
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    train_acc = []
    test_acc = []
    temp_train_acc = []
    temp_test_acc = []
    trees_grid = grid_list
    
    # Обучаем на тренировочном датасете
    for ntrees in trees_grid:
        _kwargs = {param:ntrees}
        rfc = alg(**_kwargs)
        temp_train_acc = []
        temp_test_acc = []
        for train_index, test_index in skf.split(X, y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            rfc.fit(X_train, y_train)
            temp_train_acc.append(rfc.score(X_train, y_train))
            temp_test_acc.append(rfc.score(X_test, y_test))
        train_acc.append(temp_train_acc)
        test_acc.append(temp_test_acc)

    train_acc, test_acc = np.asarray(train_acc), np.asarray(test_acc)
    print("Best accuracy on CV is {:.2f}% with {} {}".format(max(test_acc.mean(axis=1))*100, 
                                                            trees_grid[np.argmax(test_acc.mean(axis=1))],
                                                            param))
    
    fig, ax = plt.subplots(figsize=(8, 4))
    ax.plot(trees_grid, train_acc.mean(axis=1), alpha=0.5, color='blue', label='train')
    ax.plot(trees_grid, test_acc.mean(axis=1), alpha=0.5, color='red', label='cv')
    ax.fill_between(trees_grid, test_acc.mean(axis=1) - test_acc.std(axis=1), test_acc.mean(axis=1) + test_acc.std(axis=1), color='#888888', alpha=0.4)
    ax.fill_between(trees_grid, test_acc.mean(axis=1) - 2*test_acc.std(axis=1), test_acc.mean(axis=1) + 2*test_acc.std(axis=1), color='#888888', alpha=0.2)
    ax.legend(loc='best')
    ax.set_ylabel("Accuracy")
    ax.set_xlabel(param)

In [ ]:
draw_accuracy_graph(DecisionTreeClassifier,'max_depth', list(range(1, 50, 10)), X_circles, y_circles)

In [ ]:
draw_accuracy_graph(DecisionTreeClassifier,'min_samples_leaf', list(range(1, 100)), X_circles, y_circles)

# Случайный лес

Задача на kaggle: https://www.kaggle.com/c/bioresponse

Данные: https://www.kaggle.com/c/bioresponse/data / https://drive.google.com/file/d/1ynddxhNnhmGgZVqdy0M4zTuD8r3C4cBP/view?usp=sharing

По данным характеристикам молекулы требуется определить, будет ли дан биологический ответ (biological response).

### Посмотрим на данные

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
bioresponse = pd.read_csv('/content/drive/My Drive/tinkoff/train.csv', header=0, sep=',')

In [ ]:
bioresponse.head()

In [ ]:
bioresponse.shape

In [ ]:
bioresponse.columns

In [ ]:
bioresponse_target = bioresponse.Activity.values
bioresponse_data = bioresponse.iloc[:, 1:]

In [ ]:
Counter(bioresponse_target)

### Обучим RandomForestClassifier

#### Посмотрим на кривые обучения для леса, где деревья небольшой глубины

In [ ]:
rf_classifier_low_depth = RandomForestClassifier(n_estimators=50, max_depth=2, random_state=42)

In [ ]:
train_sizes, train_scores, test_scores = model_selection.learning_curve(rf_classifier_low_depth, 
                                               bioresponse_data, 
                                               bioresponse_target,
                                               train_sizes=np.arange(0.1, 1., 0.2), 
                                               cv=3, 
                                               scoring='accuracy')

In [ ]:
print(train_sizes)
print(train_scores.mean(axis = 1))
print(test_scores.mean(axis = 1))

In [ ]:
plt.grid(True)
plt.plot(train_sizes, train_scores.mean(axis = 1), 'g-', marker='o', label='train')
plt.plot(train_sizes, test_scores.mean(axis = 1), 'r-', marker='o', label='test')
plt.ylim((0.0, 1.05))
plt.legend(loc='lower right');

#### Посмотрим на кривые обучения для леса, где деревья глубже

In [ ]:
rf_classifier = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=42)

In [ ]:
train_sizes, train_scores, test_scores = model_selection.learning_curve(rf_classifier, 
                                               bioresponse_data, 
                                               bioresponse_target,
                                               train_sizes=np.arange(0.1, 1., 0.2), 
                                               cv=3, 
                                               scoring='accuracy')

In [ ]:
plt.grid(True)
plt.plot(train_sizes, train_scores.mean(axis = 1), 'g-', marker='o', label='train')
plt.plot(train_sizes, test_scores.mean(axis = 1), 'r-', marker='o', label='test')
plt.ylim((0.0, 1.05))
plt.legend(loc='lower right');

#### Самостоятельная работа: посмотрим на кривые обучения для леса, где деревья неглубокие, но их много

In [ ]:
### Ваш код здесь 

# Ансамбли – делаем случайный лес сами

## Посмотрим на данные

Возьмем известный нам датасет с цифрами

In [ ]:
digits = load_digits()
X = digits.data
y = digits.target

In [ ]:
X[0]

In [ ]:
y

## Самостоятельная работа

Для оценки качества далее нужно будет использовать cross_val_score из sklearn.cross_validation с параметром cv=10. Эта функция реализует k-fold cross validation c k равным значению параметра cv. Мы предлагаем использовать k=10, чтобы полученные оценки качества имели небольшой разброс, и было проще проверить полученные ответы. На практике же часто хватает и k=5. 

Функция cross_val_score будет возвращать numpy.ndarray, в котором будет k чисел - качество в каждом из k экспериментов k-fold cross validation. Для получения среднего значения (которое и будет оценкой качества работы) вызовите метод .mean() у массива, который возвращает cross_val_score.

### Создайте DecisionTreeClassifier с настройками по умолчанию и измерьте качество его работы с помощью cross_val_score. 

In [ ]:
### Ваш код здесь

### Воспользуйтесь sklearn.ensemble BaggingClassifier и обучите его над DecisionTreeClassifier. 

Используйте в BaggingClassifier параметры по умолчанию, задав только количество деревьев равным 100.

Обратите внимание, как соотносится качество работы композиции решающих деревьев с качеством работы одного решающего дерева.

In [ ]:
### Ваш код здесь

#### Используйте только $\sqrt{d}$ признаков

Теперь изучите параметры BaggingClassifier и выберите их такими, чтобы каждый базовый алгоритм обучался не на всех $d$ признаках, а на  $\sqrt{d}$случайных признаков. 

Корень из числа признаков - часто используемая эвристика в задачах классификации, в задачах регрессии же часто берут число признаков, деленное на три. Но в общем случае ничто не мешает вам выбирать любое другое число случайных признаков.

In [ ]:
### Ваш код здесь

#### Попробуйте выбирать случайные признаки не один раз на все дерево, а при построении каждой вершины дерева

Сделать это несложно: нужно убрать выбор случайного подмножества признаков в BaggingClassifier и добавить его в DecisionTreeClassifier. Какой параметр за это отвечает, можно понять из документации sklearn, либо просто попробовать угадать (скорее всего, у вас сразу получится). Попробуйте выбирать опять же $\sqrt{d}$ признаков. 

In [ ]:
### Ваш код здесь